In [7]:
from utils import merge_tickers_from_market_indexes, get_tickers_from_market_index

ibov = get_tickers_from_market_index("./assets/IBOV-composition.csv")
idiv = get_tickers_from_market_index("./assets/IDIV-composition.csv")

brazilian_companies = merge_tickers_from_market_indexes(ibov, idiv)

len(brazilian_companies)

112

In [9]:
brazilian_companies = [f"{ticker}.SA" for ticker in brazilian_companies]
brazilian_companies

['KLBN11.SA',
 'BBDC3.SA',
 'ITSA4.SA',
 'GOAU4.SA',
 'CSNA3.SA',
 'TOTS3.SA',
 'BPAC11.SA',
 'FLRY3.SA',
 'CPLE6.SA',
 'GGBR4.SA',
 'BBDC4.SA',
 'ELET6.SA',
 'TGMA3.SA',
 'ALPA4.SA',
 'BRAP4.SA',
 'EGIE3.SA',
 'USIM5.SA',
 'CPLE3.SA',
 'VALE3.SA',
 'JHSF3.SA',
 'HYPE3.SA',
 'UNIP6.SA',
 'RDOR3.SA',
 'MRFG3.SA',
 'CSAN3.SA',
 'DXCO3.SA',
 'RAIZ4.SA',
 'COGN3.SA',
 'SMTO3.SA',
 'PETR3.SA',
 'CMIG3.SA',
 'VAMO3.SA',
 'RANI3.SA',
 'DIRR3.SA',
 'CIEL3.SA',
 'GRND3.SA',
 'RRRP3.SA',
 'HAPV3.SA',
 'ASAI3.SA',
 'ELET3.SA',
 'ITUB4.SA',
 'ALUP11.SA',
 'SOMA3.SA',
 'TRIS3.SA',
 'NTCO3.SA',
 'PETR4.SA',
 'MULT3.SA',
 'CCRO3.SA',
 'RAIL3.SA',
 'SANB11.SA',
 'CXSE3.SA',
 'FESA4.SA',
 'TASA4.SA',
 'LEVE3.SA',
 'GOLL4.SA',
 'LAVV3.SA',
 'RENT3.SA',
 'BRFS3.SA',
 'WIZC3.SA',
 'YDUQ3.SA',
 'IRBR3.SA',
 'TRPL4.SA',
 'CRFB3.SA',
 'CPFE3.SA',
 'TIMS3.SA',
 'AGRO3.SA',
 'EQTL3.SA',
 'TAEE11.SA',
 'CVCB3.SA',
 'PETZ3.SA',
 'BEEF3.SA',
 'CSMG3.SA',
 'B3SA3.SA',
 'CMIG4.SA',
 'IGTI11.SA',
 'EMBR3.SA',
 'CYRE

## Obter os dados de dividendos

In [26]:
import yfinance as yf

tickers = yf.Tickers(["VALE3.SA", "BBAS3.SA", "PETR4.SA"]).tickers

In [38]:
import pandas as pd

all_dividends = pd.DataFrame()

for ticker, data in tickers.items():
    per_month_df = data.actions.groupby(pd.Grouper(freq="M")).sum()
    dividend_df = per_month_df.iloc[:, 0].to_frame()
    dividend_df.rename(columns={'Dividends': ticker}, inplace=True)

    if all_dividends.empty:
        all_dividends = dividend_df
        continue
    
    all_dividends = all_dividends.merge(dividend_df, left_index=True, right_index=True, how='outer')
    all_dividends.fillna(0, inplace=True)

all_dividends.reset_index(inplace=True)
all_dividends.set_index("Date", inplace=True)
all_dividends.index = pd.to_datetime(all_dividends.index)

all_dividends

,VALE3.SA,BBAS3.SA,PETR4.SA
Date,,,
2004-08-31 00:00:00-03:00,0.000000,0.000000,0.000000
2004-09-30 00:00:00-03:00,0.000000,0.000000,0.000000
2004-10-31 00:00:00-03:00,0.000000,0.000000,0.000000
2004-11-30 00:00:00-02:00,0.000000,0.000000,0.000000
2004-12-31 00:00:00-02:00,0.000000,0.000000,0.000000
...,...,...,...
2023-05-31 00:00:00-03:00,0.000000,0.000000,0.000000
2023-06-30 00:00:00-03:00,0.000000,0.993051,1.893576
2023-07-31 00:00:00-03:00,0.000000,0.000000,0.000000


In [40]:
all_dividends.to_csv("./assets/brazilian_companies_dividends.csv")

In [46]:
import plotly.express as px

fig = px.line(all_dividends, x=all_dividends.index, y=all_dividends.columns)
fig.update_layout(title="Dividendo das empresas listados em bolsa",
                  xaxis_title="Data",
                  yaxis_title="Valor")
fig.show()

/home/gabriel/Projects/3A-ai/.venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Previsão de dividendos